In [ ]:
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.interval_list import IntervalList
from rekall.merge_ops import payload_plus
from rekall.payload_predicates import payload_satisfies
from rekall.temporal_predicates import overlaps
from rekall.parsers import in_array, merge_dict_parsers, bbox_payload_parser, dict_payload_parser
from esper.caption_metadata import caption_metadata_for_video
from esper.captions import get_all_segments
videos = Video.objects.filter(id__in=[28]).all()
    
# Annotate face rows with start and end frames and the video ID
faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
    min_frame=F('face__frame__number'),
    max_frame=F('face__frame__number'),
    video_id=F('face__frame__video_id'),
    character_name=F('characteractor__character__name')
).filter(video_id__in=[v.id for v in videos])

frames_with_identity = VideoIntervalCollection.from_django_qs(
    faces_with_character_actor_qs,
    with_payload=in_array(
        dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
    )
).coalesce(payload_merge_op=payload_plus)

def convert_to_frames(intrvllist_seconds, fps):
    return intrvllist_seconds.map(
        lambda intrvl: (int(intrvl.start * fps), int(intrvl.end * fps), intrvl.payload)
    )

all_segments = get_all_segments([video.id for video in videos])
all_segments = get_all_segments([video.id for video in videos])
captions_interval_collection = VideoIntervalCollection({
    video: convert_to_frames(IntervalList(intervals), Video.objects.get(id=video).fps)
    for video, intervals in all_segments
}).dilate(10).coalesce(payload_merge_op = payload_plus).dilate(-10).map(
    lambda intrvl: (intrvl.start, intrvl.end, ' '.join(intrvl.payload))
)

In [53]:
from esper.supercuts import *
from query.models import Shot, Object, PoseMeta
from rekall.temporal_predicates import overlaps

In [54]:
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.interval_list import IntervalList
from rekall.merge_ops import payload_plus
from rekall.payload_predicates import payload_satisfies
from rekall.temporal_predicates import overlaps
from rekall.parsers import in_array, merge_dict_parsers, bbox_payload_parser, dict_payload_parser
from esper.caption_metadata import caption_metadata_for_video
from esper.captions import get_all_segments
from esper.prelude import esper_widget
import time
from IPython.display import display

In [89]:
def show_query(video_id, antagonist_name):
    videos = Video.objects.filter(id__in=[video_id]).all()
    
    # Annotate face rows with start and end frames and the video ID
    faces_with_character_actor_qs = FaceCharacterActor.objects.annotate(
        min_frame=F('face__frame__number'),
        max_frame=F('face__frame__number'),
        video_id=F('face__frame__video_id'),
        character_name=F('characteractor__character__name')
    ).filter(video_id__in=[v.id for v in videos])

    frames_with_identity = VideoIntervalCollection.from_django_qs(
        faces_with_character_actor_qs,
        with_payload=in_array(
            dict_payload_parser(VideoIntervalCollection.django_accessor, { 'character': 'character_name' }),
        )
    ).coalesce(payload_merge_op=payload_plus)
    
    def convert_to_frames(intrvllist_seconds, fps):
        return intrvllist_seconds.map(
            lambda intrvl: (int(intrvl.start * fps), int(intrvl.end * fps), intrvl.payload)
        )
    
    all_segments = get_all_segments([video.id for video in videos])
    captions_interval_collection = VideoIntervalCollection({
        video: convert_to_frames(IntervalList(intervals), Video.objects.get(id=video).fps)
        for video, intervals in all_segments
    }).dilate(10).coalesce(payload_merge_op = payload_plus).dilate(-10).map(
        lambda intrvl: (intrvl.start, intrvl.end, ' '.join(intrvl.payload))
    )
    
    frames_with_antagonist = frames_with_identity.filter(
        lambda intrvl: any(antagonist_name in char['character'] for char in intrvl.payload)
    )
    
    shots_qs = Shot.objects.filter(
        video_id=video_id,
        cinematic=True)
    shots = VideoIntervalCollection.from_django_qs(shots_qs)
    
    antagonist_shots = shots.filter_against(frames_with_antagonist, predicate=overlaps())
    
    
    result = intrvllists_to_result_with_objects(antagonist_shots, lambda a, b: [])
    widget = esper_widget(result, jupyter_keybindings=True)
    display(widget)
    return widget, result

In [94]:
widget, result = show_query(186, 'padmé')

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [95]:
selected_segments_villain = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['max_frame'])
    for i in widget.selected
]

print(selected_segments_villain)

[(39361, 39421)]


In [96]:
# ESTABLISHING SHOTS
def show_query_establishing(video_id):
    videos = Video.objects.filter(id__in=[video_id]).all()
    
    faces = Face.objects.annotate(
        min_frame=F('frame__number'),
        max_frame=F('frame__number'),
        video_id=F('frame__video_id')).filter(video_id__in=[v.id for v in videos])
    
    frames_with_faces = VideoIntervalCollection.from_django_qs(
        faces
    ).coalesce(payload_merge_op=payload_plus)
    
    
    objects = Object.objects.annotate(
        min_frame=F('frame__number'),
        max_frame=F('frame__number'),
        video_id=F('frame__video_id')).filter(video_id__in=[v.id for v in videos])
    
    frames_with_objects = VideoIntervalCollection.from_django_qs(
        objects
    ).coalesce(payload_merge_op=payload_plus)
    
    pose_meta_qs = PoseMeta.objects.annotate(
        min_frame=F('frame__number'),
        max_frame=F('frame__number'),
        video_id=F('frame__video_id')).filter(video_id__in=[v.id for v in videos])
    
    frames_with_poses = VideoIntervalCollection.from_django_qs(
        pose_meta_qs
    ).coalesce(payload_merge_op=payload_plus)
    
    
    shots_qs = Shot.objects.filter(
        video_id=video_id,
        cinematic=True)
    shots = VideoIntervalCollection.from_django_qs(shots_qs)
    
    face_shots = shots.filter_against(frames_with_faces, predicate=overlaps())
    object_shots = shots.filter_against(frames_with_objects, predicate=overlaps())
    pose_shots = shots.filter_against(frames_with_poses, predicate=overlaps())
    
    no_face_shots = shots.minus(face_shots)
    no_object_shots = shots.minus(object_shots)
    no_pose_shots = shots.minus(pose_shots)
    
    nofaceobj = no_face_shots.filter_against(no_object_shots, predicate=overlaps())
    
    establishing_shots = nofaceobj.filter_against(no_pose_shots, predicate=overlaps())
    
    establishing_shots = establishing_shots.filter_length(min_length=120)
    
    result = intrvllists_to_result_with_objects(establishing_shots, lambda a, b: [])
    #restult = intrvllists_to_result_bbox(frames_with_objects.get_allintervals())
    widget = esper_widget(result, jupyter_keybindings=True)
    display(widget)
    return widget, result

In [84]:
ewidget, result = show_query_establishing(186)

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

In [85]:
selected_segments_establishing = [
    (result['result'][i]['elements'][0]['min_frame'], result['result'][i]['elements'][0]['max_frame'])
    for i in ewidget.selected
]

print(selected_segments_establishing)

[(2846, 3012), (7596, 7765), (16250, 16382), (116803, 116957), (124395, 124683), (148939, 149106)]


In [71]:
    
videos = Video.objects.filter(id__in=[186]).all()


faces = Face.objects.annotate(
    min_frame=F('frame__number'),
    max_frame=F('frame__number'),
    video_id=F('frame__video_id')).filter(video_id__in=[v.id for v in videos])

frames_with_faces = VideoIntervalCollection.from_django_qs(
    faces
).coalesce(payload_merge_op=payload_plus)    

def convert_to_frames(intrvllist_seconds, fps):
        return intrvllist_seconds.map(
            lambda intrvl: (int(intrvl.start * fps), int(intrvl.end * fps), intrvl.payload)
        )

all_segments = get_all_segments([video.id for video in videos])
captions_interval_collection = VideoIntervalCollection({
    video: convert_to_frames(IntervalList(intervals), Video.objects.get(id=video).fps)
    for video, intervals in all_segments
}).dilate(10).coalesce(payload_merge_op = payload_plus).dilate(-10).map(
    lambda intrvl: (intrvl.start, intrvl.end, ' '.join(intrvl.payload))
)
    

    

In [101]:
captions_interval_collection.filter(lambda intrvl: 'war' in intrvl.payload.lower()).get_intervallist(186).get_intervals() 
#.get_intervallist(74).get_intervals()

#.map - map payloads to all be ov

[<Interval start:18167 end:18197 payload:I 've been looking forward to this>,
 <Interval start:35852 end:36519 payload:Chancellor Palpatine are you all right Yes Thanks to your two Jedi knights They killed Count Dooku but General Grievous has escaped once again General Grievous will run and hide as he always does He 's a coward But with Count Dooku dead he is the leader of the droid army And I assure you the senate will vote to continue the war as long as Grievous is alive Then the Jedi Council will make finding Grievous our highest priority>,
 <Interval start:41458 end:41532 payload:The end of the war is near General>,
 <Interval start:50799 end:50840 payload:It 'll make it easier for us to end this war>,
 <Interval start:56745 end:56792 payload:But that 's treason We are at war Anakin>,
 <Interval start:59681 end:59739 payload:I think this war is destroying the principles of the Republic>,
 <Interval start:60291 end:60343 payload:This war represents a failure to listen>,
 <Interval s

In [68]:
captions_interval_collection.get_intervallist(186).get_intervals()

[<Interval start:4505 end:4528 payload:Lock onto him R2>,
 <Interval start:4549 end:4726 payload:Master General Grievous 's ship is directly ahead the one crawling with vulture droids I see it Oh this is going to be easy>,
 <Interval start:4935 end:5128 payload:Oddball do you copy Copy Red Leader Mark my position Form your squad up behind me We 're on your tail General Kenobi Set S foils in attack position>,
 <Interval start:5475 end:5547 payload:This is where the fun begins Let them pass between us>,
 <Interval start:5982 end:5988 payload:They 're all>,
 <Interval start:6084 end:6094 payload:over me Get them out>,
 <Interval start:6115 end:6167 payload:No They are doing their job so we can do ours>,
 <Interval start:6520 end:6545 payload:Missiles Pull up>,
 <Interval start:6621 end:6664 payload:They overshot us They 're coming around>,
 <Interval start:6826 end:6855 payload:All right R4 No no>,
 <Interval start:6878 end:6890 payload:Nothing too>,
 <Interval start:6992 end:7049 payload